In [1]:
print('start')

start


In [4]:
import glob
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import re
from DataProcessing import *
from DataImporting import sequence_list
peak_list, residue_list = main_data_processing()

In [15]:
two_d_dict={}
two_d_dict['HShift'] = [x.get_data('TROSYHShift') for x in peak_list]
two_d_dict['NShift'] = [x.get_data('TROSYNShift') for x in peak_list]
two_d_dict['CA'] = [x.get_data('CAShift') for x in peak_list]
two_d_dict['CB'] = [x.get_data('CBShift') for x in peak_list]
two_d_dict['CAPrime'] = [x.get_data('CAPrimeShift') for x in peak_list]
two_d_dict['CBPrime'] = [x.get_data('CBPrimeShift') for x in peak_list]
two_d_dataframe = pd.DataFrame(two_d_dict)
print(two_d_dataframe.head())

closest_ca_list, closest_cb_list = [], []
for row1 in range(len(two_d_dataframe)):
    ca_shift = two_d_dataframe.loc[row1, "CA"]
    cb_shift = two_d_dataframe.loc[row1, "CB"]
    if np.isnan(ca_shift):
        closest_ca_list.append(None)
    else:
        ls_ca = []
        for row2 in range(len(two_d_dataframe)):
            ca_prime_shift = two_d_dataframe.loc[row2, "CAPrime"]
            if not np.isnan(ca_prime_shift):
                ls_ca.append(round(abs(ca_shift - ca_prime_shift), 4))
        ls_ca.sort()
        closest_ca_list.append(ls_ca[0:5])

    if np.isnan(cb_shift):
        closest_cb_list.append(None)
    else:
        ls_cb = []
        for row2 in range(len(two_d_dataframe)):
            cb_prime_shift = two_d_dataframe.loc[row2, "CBPrime"]
            if not np.isnan(cb_prime_shift):
                ls_cb.append(round(abs(cb_shift - cb_prime_shift), 4))
        ls_cb.sort()
        closest_cb_list.append(ls_cb[0:5])

# print(closest_ca_list[0:3])
# print(closest_cb_list[0:3])
two_d_dataframe['ClosestCAShift1D'] = closest_ca_list
two_d_dataframe['ClosestCBShift1D'] = closest_cb_list

closest_2d_list = []
row_number = []
for row1 in range(len(two_d_dataframe)):
    ca_shift = two_d_dataframe.loc[row1, "CA"]
    cb_shift = two_d_dataframe.loc[row1, "CB"]
    if np.isnan(ca_shift):
        closest_2d_list.append(None)
    elif np.isnan(cb_shift):
        closest_2d_list.append(None)
    else:
        ls = []
        ls1 = []
        for row2 in range(len(two_d_dataframe)):
            ca_prime_shift = two_d_dataframe.loc[row2, "CAPrime"]
            cb_prime_shift = two_d_dataframe.loc[row2, "CBPrime"]
            if not np.isnan(ca_prime_shift) and not np.isnan(cb_prime_shift):
                ls1.append(row2)
                ca_delta_squared = (abs(ca_shift - ca_prime_shift)) ** 2
                cb_delta_squared = (abs(cb_shift - cb_prime_shift)) ** 2
                ls.append(((ca_delta_squared + cb_delta_squared)/2) ** .5)
        ls, row_number = zip(*sorted(zip(ls, ls1)))
        closest_2d_list.append([round(x, 5) for x in ls[0:5]])
        row_number.append(ls1[0:5])
two_d_dataframe['Closest2DShift (ppm)'] = closest_2d_list
two_d_dataframe['Corresponding peak'] = row_number


   HShift   NShift      CA      CB  CAPrime  CBPrime
0   5.756  114.300  58.590  57.296   60.265      NaN
1   5.251  126.472  55.967     NaN      NaN      NaN
2   6.303  122.611  49.834  69.595   49.177   66.158
3   6.425  120.326  56.740  55.626   58.271   22.790
4   6.285  109.235  63.646  56.981   63.742   57.774


AttributeError: 'tuple' object has no attribute 'append'

In [8]:
two_d_dataframe.head()

,HShift,NShift,CA,CB,CAPrime,CBPrime,ClosestCAShift1D,ClosestCBShift1D,Closest2DShift (ppm)
0,5.756,114.300,58.590,57.296,60.265,NaN,"[0.089, 0.134, 0.178, 0.229, 0.231]","[0.014, 0.022, 0.05, 0.058, 0.18]","[0.14644, 0.17141, 0.29855, 0.52331, 0.5289]"
1,5.251,126.472,55.967,NaN,NaN,NaN,"[0.018, 0.053, 0.067, 0.081, 0.089]",None,None
2,6.303,122.611,49.834,69.595,49.177,66.158,"[0.09, 0.105, 0.33, 0.657, 0.884]","[0.307, 0.557, 0.587, 0.614, 0.775]","[0.72084, 1.68694, 1.90425, 1.97339, 2.02967]"
3,6.425,120.326,56.740,55.626,58.271,22.790,"[0.024, 0.041, 0.089, 0.164, 0.173]","[0.018, 0.021, 0.024, 0.087, 0.151]","[0.57374, 0.59421, 0.60949, 0.62499, 0.63297]"
4,6.285,109.235,63.646,56.981,63.742,57.774,"[0.014, 0.035, 0.04, 0.07, 0.075]","[0.068, 0.073, 0.102, 0.122, 0.128]","[0.49359, 0.52464, 0.56483, 0.7701, 0.77191]"


In [14]:
print(two_d_dataframe.iloc[:30, -3:])

                       ClosestCAShift1D                     ClosestCBShift1D  \
0   [0.089, 0.134, 0.178, 0.229, 0.231]    [0.014, 0.022, 0.05, 0.058, 0.18]   
1   [0.018, 0.053, 0.067, 0.081, 0.089]                                 None   
2     [0.09, 0.105, 0.33, 0.657, 0.884]  [0.307, 0.557, 0.587, 0.614, 0.775]   
3   [0.024, 0.041, 0.089, 0.164, 0.173]  [0.018, 0.021, 0.024, 0.087, 0.151]   
4     [0.014, 0.035, 0.04, 0.07, 0.075]  [0.068, 0.073, 0.102, 0.122, 0.128]   
5   [0.001, 0.006, 0.027, 0.029, 0.034]  [0.045, 0.053, 0.059, 0.067, 0.071]   
6   [0.016, 0.038, 0.087, 0.087, 0.107]  [0.096, 0.109, 0.241, 0.257, 0.434]   
7     [0.01, 0.054, 0.06, 0.081, 0.088]   [0.187, 0.63, 0.695, 0.924, 0.944]   
8   [0.041, 0.048, 0.049, 0.058, 0.139]  [0.002, 0.033, 0.194, 0.199, 0.236]   
9    [0.025, 0.057, 0.077, 0.126, 0.18]   [0.29, 0.539, 0.592, 0.853, 0.887]   
10  [0.061, 0.359, 0.374, 0.388, 1.153]    [0.012, 0.038, 0.086, 0.09, 0.16]   
11  [0.018, 0.027, 0.105, 0.122, 0.209] 